# Learning Eikon Data API - Internship Diary(Part 2)

This is the second part of my learning diary. In this part, I use data from Eikon Data API to create interactive widgets and demonstrate how to use Screener with Eikon Data API. Please refer to Learning Eikon Data API - Internship Diary (Part 1) for the first part. 

First, I import Python packages used by the application.

- **[Cufflinks](https://plotly.com/python/pandas-backend/#what-about-cufflinks)** :Connecting Plotly with pandas to create graphs and charts of data frames directly.
- **[pandas](https://pandas.pydata.org/getting_started.html)** : The fast, powerful, flexible, and easy to use open-source data analysis and manipulation tool.
- **[NumPy](https://numpy.org/install/)** : The fundamental package for scientific computing in Python.
- **[Plotly](https://plotly.com/python/getting-started/)** : An interactive, open-source plotting library. 
- **[ipywidgets](https://ipywidgets.readthedocs.io/en/latest/)** :  An interactive HTML widgets for Jupyter notebooks, JupyterLab, and the IPython kernel. 
- **[IPython.display](https://dateutil.readthedocs.io/en/stable/relativedelta.html)** : Public API for display tools in IPython.
- **[datetime](https://docs.python.org/2/library/datetime.html#module-datetime)** : Supplies classes for manipulating dates and times. 
- **[relativedelta](https://dateutil.readthedocs.io/en/stable/relativedelta.html)**: It is designed to be applied to an existing datetime and can replace specific components of that datetime, or represents an interval of time.

In [1]:
import eikon as ek  #Eikon Python wrapper package
import cufflinks as cf 
import pandas as pd 
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets #IPython widgets 
from ipywidgets import Button, HBox, VBox
from IPython.display import display, clear_output
from datetime import datetime, timedelta 
from dateutil.relativedelta import relativedelta 

The following Python and package versions are used.

In [2]:
import sys 
print(sys.version) #python version

3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)]


In [3]:
ek.__version__ #eikon version

'1.1.2'

In [4]:
cf.__version__ #cufflinks version

'0.17.3'

In [5]:
pd.__version__ #pandas version

'1.0.1'

In [6]:
np.__version__ #numpy version

'1.18.1'

In [7]:
from plotly import __version__ 
print(__version__) #plotly version

4.8.2


In [8]:
widgets.__version__ #ipywidgets version

'7.5.1'

Next, connect to Eikon Data API by setting the App Key. 

In [9]:
ek.set_app_key('[APP_KEY]') #connecting to Eikon Data API

**Note:**

- Please change <APP_KEY> to your App Key. For more information, please refer to [Create an App Key for your application](https://developers.refinitiv.com/eikon-apis/eikon-data-api/quick-start#create-app-key).
- Before you run the application, please make sure that **Refinitiv Workspace** or **Eikon Desktop** is running.

## Time Series Chart Widget
In this section, I will create an interactive widget that plots time-series data returned by the `ek.get_timeseries()` method on a line chart. The widget accepts parameters, such as RICs, fields, interval, start date, and end date from users. Then, it calls the `ek.get_timeseries()` method with the retrieved parameters to get the time-series data. Finally, it uses the time-series data to create an interactive line chart that compares the stock prices of each RIC. 

The code below shows how the widget is implemented.


In [10]:
import warnings
warnings.filterwarnings('ignore') #prevent warning message 
cf.set_config_file(offline=True) #set the plotting mode to offline

class WidgetPlot:
    #widget for input RICs 
    ts_get_rics = widgets.Text(
        value = 'AAPL.O;MSFT.O',
        placeholder = 'Input RICs',
        description = 'RICs:',
        disabled = False   
    )

    #widget for select start date 
    ts_get_start = widgets.DatePicker(
        value = datetime.now() - relativedelta(years = 1),
        description = 'Start Date :', 
        disabled = False
    )

    #widget for select  end date
    ts_get_end = widgets.DatePicker(
        value = datetime.now(),
        description = 'End Date :',
        disabled = False
    )

    #widget for select interval
    ts_get_interval = widgets.Dropdown(
        value = 'daily',
        placeholder = 'Choose Interval',
        options = ['tick', 'minute', 'hour', 'daily', 'weekly', 'monthly', 'quarterly', 'yearly'],
        description = 'Interval:',
        ensure_option = True,
        disabled = False
    )

    # widget for input Fields 
    ts_get_fields = widgets.Dropdown(
        value = 'CLOSE',
        placeholder = 'Choose Fields',
        options = ['VOLUME', 'HIGH', 'LOW', 'OPEN', 'CLOSE', 'COUNT'],
        description = 'Fields:',
        ensure_option = True,
        disabled = False
    ) 

    #widgets arrangement 
    left_box = widgets.VBox([ts_get_rics, ts_get_interval, ts_get_fields])
    right_box = widgets.VBox([ts_get_start, ts_get_end])
    
    #display the widgets
    display(HBox([left_box, right_box]))
    
    #widget button for submit the input
    ts_button = widgets.Button(description='Run') 
    ts_output = widgets.Output()

    def __init__(self):
        display(self.ts_button, self.ts_output)
        self.ts_button.on_click(self.on_button_clicked)  
        self.ts_output
        self.ts_get_fields.value
        self.ts_get_interval.value
        
    def on_button_clicked(self,b):
        with self.ts_output:
            self.ts_output.clear_output() #clear output
            st = self.ts_get_start.value #get start date value from datepicker
            ed = self.ts_get_end.value #get end date value from datepicker
            start = st.strftime('%Y-%m-%d %H:%M:%S') #convert start date to yyyy-mm-dd format
            end = ed.strftime('%Y-%m-%d %H:%M:%S') #convert end date to yyyy-mm-dd format
            rics = self.ts_get_rics.value.split(";") #retrieve each ric seperated by ';'
            
            #get timeseries data
            data = ek.get_timeseries(rics,
                                            start_date = start,
                                            end_date = end,
                                            fields = self.ts_get_fields.value,
                                            interval = self.ts_get_interval.value)
            #check if variables is exist
            if data.isnull().values.all():
                print('\033[31m' + 'Error, value not available(NaN value detected).')
                return
            else:
                def multi_plot(data, addAll = True):
                    fig = go.Figure()
                    fig.update_layout(legend_title_text = data.columns.name, 
                                        title_text = 'Time Series with Range Slider and Selectors')
                    num = len(rics) #check number of RICs 

                    #button of dropdown menu
                    button_all = dict(label = 'All',
                                        method = 'update',
                                        args = [{'visible': data.columns.isin(data.columns),
                                                'title': 'All',
                                                'showlegend':True}])

                    def create_layout_button(column):
                        return dict(label = column,
                                    method = 'update',
                                    args = [{'visible': data.columns.isin([column]),
                                            'title': column,
                                            'showlegend': True}])
                    #add dropdown for more than 2 RICs
                    if num > 1:
                        fig.update_layout(
                            updatemenus = [
                                go.layout.Updatemenu(
                                    pad = {"r": 10, "t": 10},
                                    y = 1.15,
                                    yanchor = "top",
                                    active = 0,
                                    buttons = ([button_all] * addAll) + list(data.columns.map(lambda column: create_layout_button(column)))
                                )
                            ]
                        )

                    #add plot 
                    for column in data.columns.to_list():
                        fig.add_trace(
                            go.Scatter(
                                x = data.index,
                                y = data[column],
                                mode ='lines',
                                name = column
                            )
                        )
                    #add range slider and selector
                    fig.update_xaxes(
                        rangeslider_visible = True,
                        rangeselector = dict(
                            buttons = list([
                                dict(count = 7, label = '1w', step = 'day', stepmode = 'backward'),
                                dict(count = 1, label = '1m', step = 'month', stepmode = 'backward'),
                                dict(count = 6, label = '6m', step = 'month', stepmode = 'backward'),
                                dict(count = 1, label = 'YTD', step = 'year', stepmode = 'todate'),
                                dict(count = 1, label = '1y', step = 'year', stepmode = 'backward'),
                                dict(step = 'all')
                            ])
                        )
                    )
                    fig.show()
                multi_plot(data)
WidgetPlot()

Button(description='Run', style=ButtonStyle())

Output()

Fill in the parameters and then click the Run button to execute the code.

#### Parameters' definitions used in the widget::
- RICs:  A single RIC or list of RICs to retrieve historical data( Use semicolon as a delimiter). 
    - `AAPL.O`: Apple stock
    - `MSFT.O`: Microsoft stock
- Interval: DTime-series data interval (tick, minute, hour, daily, weekly, monthly, quarterly, yearly).
    - `Daily` : Daily interval  
- Fields: Use this parameter to filter the returned fields set.
    - `CLOSE`: Closing price
- Start Date : Starting date(mm/dd/yyyy) of the historical range. 
- End Date: End date(mm/dd/yyyy) of the historical range.

After executing the code, the line chart will be displayed, as shown below. 

## Fundamental Table Widget

In this section, I will create a widget that displays fundamental data returned by the `ek.get_data()` method in a table. The widget accepts RICs and fields as parameters. Then, it calls the `ek.data()` method with the retrieved parameters to get the fundamental data. Finally, it displays the result in a table format. 

**Tip:** You can use the **Data Item Browser (DIB)** to explore data items by search and configure supported parameters. For detailed information, you can visit the [Data Item Browser (DIB) Overview](https://developers.refinitiv.com/article/how-discover-available-fields-data-grid-service-jetapp-studio-html5-sdk-api-using-eikon) article.


In [11]:
class WidgetTable:
    
    #for input RICs 
    get_rics = widgets.Textarea(
        value = 'TWTR.K;FB.OQ;MSFT.OQ;GOOGL.OQ;NFLX.OQ',
        placeholder = 'Input RICs',
        description = 'RICs:',
        disabled = False   
    )

    #for input Fields 
    get_fields = widgets.Textarea(
        value = 'TR.CompanyName;TR.PE;TR.TotalDebtToEV;TR.EVToSales',
        placeholder = 'Input Fields',
        description = 'Fields:',
        disabled = False
    )
    
    #display the widgets
    display(HBox([get_rics, get_fields]))
    
    #button for submit the input
    button = widgets.Button(description='Run')
    output = widgets.Output()
    
    def __init__(self):
        display(self.button, self.output)
        self.button.on_click(self.on_button_clicked)  
        self.output
        
    def on_button_clicked(self,c):
        with self.output:
            self.output.clear_output() #clear and update output
            rics = self.get_rics.value.split(";") #retrieve each ric seperated by ';'
            fields = self.get_fields.value.split(";") #retrieve each field seperated by ';'
            
            #get data
            df, err = ek.get_data(rics, fields)
            
            #check if variables is exist
            if df.iloc[:,1:].isnull().values.all():
                print('\033[31m' + 'Error, invalid input.')
                return
            else:
                data = df.round(decimals = 3) #limiting floats to three decimal points
                #add table
                fig = go.Figure(
                        data = [go.Table(
                        header = dict(values = df.columns,
                                    font_size = 13,
                                    height = 30),
                        cells = dict(values = data.T,
                                    font_size = 13,
                                    height = 30))
                        ])
                
                #table layout
                fig.update_layout(
                    title_text = 'Fundamental Data',
                    autosize = True,
                    margin = dict(
                        l = 0,
                        r = 0,
                        b = 0,
                        t = 50,
                    )
                )
                fig.show()
            #add text to show error
            if err != None:
                print('Error:')
                print(err, sep = "\n")
                    
WidgetTable()

Button(description='Run', style=ButtonStyle())

Output()

Fill in RICs and Fields and then click the Run button to execute the code. 
#### Parameters' definitions used in the widget:
- RICs: A single RIC or list of RICs to retrieve data (Use semicolon as a delimiter). For example:
    - `TWTR.K`: Twitter stock
    - `FB.OQ`: Facebook stock
    - `MSFT.OQ`: Microsoft stock
    - `GOOGL.OQ`: Google stock
    - `NFLX.OQ`: Netflix stock
- Fields: List of fields to request (Use semicolon as a delimiter). For example:
    - `TR.PE`: Price-to-Earnings Ratio (Daily Times Series Ratio). A valuation ratio of a company's current share price relative to its EPS. EPS is LTM Earnings per Share From Continuing Operations. PE is not calculated when LTM EPS is less than or equal to Zero.
    - `TR.TotalDebtToEV`: Total Dept To Enterprise Value (Daily Times Series Ratio). Total Debt includes Short and Long term Debt for the most recent fiscal period. For Banks, Cash and Due from Banks is considered in place of Cash and Short Term Investments.
    - `TR.EVToSales`: Enterprise Value To Sales (Daily Times Series Ratio). EV to Sales ratio is not calculated when LTM Sales is less than or equal to Zero. For Banks, Interest Income plus Non-Interest Income is considered as Total Sales and Cash and Due from Banks is considered in place of Cash and Short Term Investments. 
    
**Note:**
- `EV` represents the sum of Market Capitalization, Total Debt, Preferred Stock and Minority Interest minus Cash and Short Term Investments for the most recent fiscal period. 
- `Market Cap` is calculated by multiplying Current Total Shares Outstanding by Latest Close Price. 
- `Sales` is Last twelve months(LTM) Total Revenue. 

### SCREENER 

With the `ek.get_data()` method, `SCREENER` can be used to filter stocks based on user-defined metrics. `SCREENER` is a flexible idea-generation tool that allows you to find securities in the investable universe that display certain characteristics and match your investment philosophy or style. For more information, please visit the [Find Your Right Companies with SCREENER](https://developers.refinitiv.com/article/find-your-right-companies-screener-eikon-data-apispython) article.

For example, I can use the `ek.get_data()` method and `SCREENER` to find companies in SET (Stock Exchange of Thailand) that are in banking and investment services and have company market capitalization greater than one billion, as shown below.

In [12]:
# screener syntax in Python format 
# In Banking and Investment Services business sector 
# And company market capitalization is greater than one billion Thai Baht 
# And listed in the Stock Exchange of Thailand 
screen_list = """SCREEN(U(IN(Equity(active,public,primary))),
                    IN(TR.TRBCBusinessSectorCode,"5510"), 
                    IN(TR.ExchangeMarketIdCode,"XBKK"), 
                    TR.CompanyMarketCap(Scale=9)>=TR.CompanyMarketCap(Scale=9), 
                    CURN=THB)"""

fields = ['TR.CompanyName', 'TR.TRBCBusinessSector', 'TR.ExchangeName', 'TR.CompanyMarketCap']

In [13]:
# get data
screen_result, err = ek.get_data(screen_list, fields)
screen_result.head(10) # show only first 10 rows

,Instrument,Company Name,TRBC Business Sector Name,Exchange Name,Company Market Cap
0,BAY.BK,Bank of Ayudhya PCL,Banking & Investment Services,STOCK EXCHANGE OF THAILAND,1.538801e+11
1,KGI.BK,KGI Securities Thailand PCL,Banking & Investment Services,STOCK EXCHANGE OF THAILAND,6.771995e+09
2,CIMBT.BK,CIMB Thai Bank PCL,Banking & Investment Services,STOCK EXCHANGE OF THAILAND,2.011965e+10
3,ASP.BK,Asia Plus Group Holdings PCL,Banking & Investment Services,STOCK EXCHANGE OF THAILAND,3.628850e+09
4,SCB.BK,Siam Commercial Bank PCL,Banking & Investment Services,STOCK EXCHANGE OF THAILAND,2.269168e+11
5,PL.BK,Phatra Leasing PCL,Banking & Investment Services,STOCK EXCHANGE OF THAILAND,1.200341e+09
6,BFIT.BK,Srisawad Finance PCL,Banking & Investment Services,STOCK EXCHANGE OF THAILAND,1.619959e+10
7,KKP.BK,Kiatnakin Bank PCL,Banking & Investment Services,STOCK EXCHANGE OF THAILAND,3.268609e+10
8,KTB.BK,Krung Thai Bank PCL,Banking & Investment Services,STOCK EXCHANGE OF THAILAND,1.371376e+11
9,MFC.BK,MFC Asset Management PCL,Banking & Investment Services,STOCK EXCHANGE OF THAILAND,1.689325e+09


Next, I will sort the results by the Company Market Capitalization and display the top ten companies that have the highest market capitalization.

In [14]:
# top 10 company sort by Company Market Capitalization 
ranking = screen_result.sort_values(by='Company Market Cap',ascending=False) 
ranking.head(10) # show only first 10 rows

,Instrument,Company Name,TRBC Business Sector Name,Exchange Name,Company Market Cap
4,SCB.BK,Siam Commercial Bank PCL,Banking & Investment Services,STOCK EXCHANGE OF THAILAND,2.269168e+11
12,KBANK.BK,Kasikornbank PCL,Banking & Investment Services,STOCK EXCHANGE OF THAILAND,1.931125e+11
11,BBL.BK,Bangkok Bank PCL,Banking & Investment Services,STOCK EXCHANGE OF THAILAND,1.892033e+11
0,BAY.BK,Bank of Ayudhya PCL,Banking & Investment Services,STOCK EXCHANGE OF THAILAND,1.538801e+11
8,KTB.BK,Krung Thai Bank PCL,Banking & Investment Services,STOCK EXCHANGE OF THAILAND,1.371376e+11
37,MTC.BK,Muangthai Capital PCL,Banking & Investment Services,STOCK EXCHANGE OF THAILAND,1.055945e+11
14,TMB.BK,TMB Bank PCL,Banking & Investment Services,STOCK EXCHANGE OF THAILAND,9.119121e+10
35,BAM.BK,Bangkok Commercial Asset Management PCL,Banking & Investment Services,STOCK EXCHANGE OF THAILAND,7.920410e+10
19,KTC.BK,Krungthai Card PCL,Banking & Investment Services,STOCK EXCHANGE OF THAILAND,7.898049e+10
36,SAWAD.BK,Srisawad Corporation PCL,Banking & Investment Services,STOCK EXCHANGE OF THAILAND,6.565918e+10


Then, I convert the Company Market Capitalization's values into a readable format. 

In [15]:
#change Company Market Cap value to billion format 
ranking['Company Market Cap(Billions)'] =  (screen_result['Company Market Cap'].astype(float)/1000000000).round(2).astype(str)
ranking.drop(['Company Market Cap'], axis='columns', inplace=True) #drop original column
ranking.head(10)

,Instrument,Company Name,TRBC Business Sector Name,Exchange Name,Company Market Cap(Billions)
4,SCB.BK,Siam Commercial Bank PCL,Banking & Investment Services,STOCK EXCHANGE OF THAILAND,226.92
12,KBANK.BK,Kasikornbank PCL,Banking & Investment Services,STOCK EXCHANGE OF THAILAND,193.11
11,BBL.BK,Bangkok Bank PCL,Banking & Investment Services,STOCK EXCHANGE OF THAILAND,189.2
0,BAY.BK,Bank of Ayudhya PCL,Banking & Investment Services,STOCK EXCHANGE OF THAILAND,153.88
8,KTB.BK,Krung Thai Bank PCL,Banking & Investment Services,STOCK EXCHANGE OF THAILAND,137.14
37,MTC.BK,Muangthai Capital PCL,Banking & Investment Services,STOCK EXCHANGE OF THAILAND,105.59
14,TMB.BK,TMB Bank PCL,Banking & Investment Services,STOCK EXCHANGE OF THAILAND,91.19
35,BAM.BK,Bangkok Commercial Asset Management PCL,Banking & Investment Services,STOCK EXCHANGE OF THAILAND,79.2
19,KTC.BK,Krungthai Card PCL,Banking & Investment Services,STOCK EXCHANGE OF THAILAND,78.98
36,SAWAD.BK,Srisawad Corporation PCL,Banking & Investment Services,STOCK EXCHANGE OF THAILAND,65.66


Otherwise, you can try out the [dataquery.py](https://github.com/Refinitiv-API-Samples/dataquery) which is designed to act as a syntax builder for the screening queries. This way Eikon Data API users no longer need to rely on Excel and generate the screening request along with various pre-processing functions directly in the Python script. For detailed information, please visit the [Dataquery – a syntax builder for screening with Eikon Data API](https://developers.refinitiv.com/article/dataquery-%E2%80%93-syntax-builder-screening-eikon-data-api) article.

## Summary

In this article, I demonstrate how to retrieve data from Eikon Data API using `ek.get_timeseries()` and `ek.get_data()`. Then use the retrieved data to create a time series plot and table using Plotly and Jupyter widgets. Lastly, I use `SCREENER` to retrieve a list of companies that matches the criteria from the defined syntax.

## Learn more

To learn more about the Eikon Data APIs Libraries, you can join the [Refinitiv Developer Community](https://developers.refinitiv.com/) by registering and login to the Refinitiv Developer Community portal. Then, you will get free access to a number of learning materials like [Quick Start guides](https://developers.refinitiv.com/eikon-apis/eikon-data-api/quick-start), [Tutorials](https://developers.refinitiv.com/eikon-data-apis/learning), [Documentation](https://developers.refinitiv.com/eikon-apis/eikon-data-api/docs), and much more

## Getting Help and Support

If you have any questions regarding the API usage, please post them on the [Eikon Data APIs Q&A Forum](https://developers.refinitiv.com/eikon-apis/eikon-data-api/qa). The Refinitiv Developer Community will be very pleased to help you.

## References

1. Eikon Data API. Refinitiv Developer Community. Retrieved Jun 18, 2020,  from https://developers.refinitiv.com/eikon-apis/eikon-data-api
2. Jupyter Widgets. ipywidgets. Retrieved Jun 18, 2020,  from https://ipywidgets.readthedocs.io/en/latest/index.html 
3. Plotly. Plotly Python Open Source Graphing Library. Retrieved Jun 18, 2020, from https://plotly.com/python/ 
4. Jintamalit, C. (2020).  Find Your Right Companies with SCREENER | Eikon Data APIs(Python). Retrieved Jul 2, 2020, from https://developers.refinitiv.com/article/find-your-right-companies-screener-eikon-data-apispython
5. Jintamalit, C. (2019). Data Item Browser (DIB) Overview in How to discover available fields for Data Grid service on JET(App Studio HTML5 SDK) API using Eikon Desktop. Retrieved Jun 18, 2020, from https://developers.refinitiv.com/article/how-discover-available-fields-data-grid-service-jetapp-studio-html5-sdk-api-using-eikon.
6. Sopotnitskiy, L. (2020). Dataquery – a syntax builder for screening with Eikon Data API. Retrieved Jul 8, 2020, from https://developers.refinitiv.com/article/dataquery-%E2%80%93-syntax-builder-screening-eikon-data-api